# Building the model

The purpose of this notebook is to build the model that will eventually be used as the model behind the PL prediction for the app, and building the script to rebuild the model when new player data is introduced weekly

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

## Loading the dataset

In [2]:
combined = pd.read_csv("../final_combined_dataframe.csv")

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 89 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          1927 non-null   int64  
 1   match_id                            1927 non-null   object 
 2   goals                               1927 non-null   float64
 3   assists                             1927 non-null   float64
 4   non_penalty_goals                   1927 non-null   float64
 5   penalties_scored                    1927 non-null   float64
 6   penalties_attempted                 1927 non-null   float64
 7   yellow_cards                        1927 non-null   float64
 8   red_cards                           1927 non-null   float64
 9   expected_goals                      1927 non-null   float64
 10  non_penalty_expected_goals          1927 non-null   float64
 11  expected_assisted_goals             1927 no

In [4]:
combined.head()

,Unnamed: 0,match_id,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,...,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
0,0,m-00381,0.907776,0.396142,0.738889,0.168887,0.236302,0.093022,-0.048172,0.776569,...,6,4,2,5,11,8,2,1,0,0
1,1,m-00382,0.155008,0.133838,0.123306,0.031702,0.039064,0.131687,-0.000329,0.200098,...,4,1,7,4,11,9,1,1,0,0
2,2,m-00383,-0.039926,-0.056540,-0.137227,0.097301,0.109960,0.066208,0.011733,-0.109844,...,6,9,5,5,9,11,1,2,0,0
3,3,m-00384,0.133878,-0.008924,0.122145,0.011733,0.011733,0.185723,-0.023049,0.212802,...,1,4,2,5,9,8,2,1,0,0
4,4,m-00385,0.401345,0.138483,0.411797,-0.010452,-0.001508,0.071065,-0.018950,0.419927,...,2,5,3,5,11,12,2,2,0,0


## Column Lists

In [5]:
output_columns = [
	"home_goals", "away_goals", "home_shots", "away_shots", "home_shots_on_target", "away_shots_on_target",
	"home_corners", "away_corners", "home_fouls", "away_fouls", "home_yellow_cards", "away_yellow_cards",
	"home_red_cards", "away_red_cards"
]

player_stats_columns = [
	"player_id", "minutes_played","ninetys","goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
pure_stats_columns = [
	"minutes_played","goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
team_stats_columns = [
	"team_id", "goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]
pure_stats_columns_no_minutes = [
	"goals","assists","non_penalty_goals","penalties_scored","penalties_attempted","yellow_cards","red_cards","expected_goals",
	"non_penalty_expected_goals","expected_assisted_goals","progressive_carries","progressive_passes","progressive_passes_received","total_passing_distance","total_progressive_passing_distance","short_passes_completed","short_passes_attempted","medium_passes_completed","medium_passes_attempted",
	"long_passes_completed","long_passes_attempted","expected_assists","key_passes","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","shots","shots_on_target","average_shot_distance","shots_from_free_kicks",
	"shots_from_penalties","touches","touches_in_defensive_penalty_area","touches_in_defensive_third","touches_in_middle_third","touches_in_attacking_third","touches_in_attacking_penalty_area","live_ball_touches","take_ons_attempted","take_ons_succeeded","times_tackled_during_take_on",
	"carries","total_carrying_distance","progressive_carrying_distance","carries_into_final_third","carries_into_penalty_area","miscontrols","dispossessed","passes_received","tackles","tackles_won","defensive_third_tackles",
	"middle_third_tackles","attacking_third_tackles","dribblers_tackled","dribbler_tackles_attempted","shots_blocked","passes_blocked","interceptions","clearances","errors_leading_to_shot","goals_against","shots_on_target_against","saves","clean_sheets","penalties_faced","penalties_allowed","penalties_saved","penalties_missed"
]

## Scaling

In [6]:
scaler = StandardScaler(copy=True)

In [7]:
combined_standardized = scaler.fit_transform(combined[pure_stats_columns_no_minutes])

In [8]:
evr = {}
n_components = {}
feature_to_pc = {}

for n in range(1,26):
	pca = PCA(n_components = n, random_state=938)
	pca.fit(combined_standardized)
	feature_to_pc_map = pd.DataFrame(pca.components_, columns=pure_stats_columns_no_minutes)
	components = pca.transform(combined_standardized)
	components_df = pd.DataFrame(data=components[:, [p for p in range(n)]], columns=pca.get_feature_names_out(), )
	
	evr[n] = sum(pca.explained_variance_ratio_)
	n_components[n] = components_df
	feature_to_pc[n] = feature_to_pc_map

pd.DataFrame(data=evr, index=["explained_variance_ratio"]).T
feature_to_pc[15].to_csv("../feature_to_15_pcs.csv")


Check the effect of different numbers if PCs on the outcome of the NN, testing n=2, 5 and 10. Use each n components to train and test the neural network, and compare the performance of each one

## Number of Features

In [9]:
INPUT_SIZE = 69
OUTPUT_SIZE = 14
ROWS = 1927

### n = 2

Train test split of data

In [10]:
n=2
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

Model definition

In [11]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500


2024-01-16 10:08:14.765064: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


31/31 [==============================] - 0s 510us/step - loss: 28.1178 - accuracy: 0.4075
Epoch 2/500
31/31 [==============================] - 0s 437us/step - loss: 16.9625 - accuracy: 0.4653
Epoch 3/500
31/31 [==============================] - 0s 393us/step - loss: 14.2166 - accuracy: 0.4698
Epoch 4/500
31/31 [==============================] - 0s 343us/step - loss: 11.3974 - accuracy: 0.4705
Epoch 5/500
31/31 [==============================] - 0s 459us/step - loss: 9.7117 - accuracy: 0.4666
Epoch 6/500
31/31 [==============================] - 0s 412us/step - loss: 9.4157 - accuracy: 0.4731
Epoch 7/500
31/31 [==============================] - 0s 400us/step - loss: 8.7409 - accuracy: 0.4705
Epoch 8/500
31/31 [==============================] - 0s 861us/step - loss: 8.6366 - accuracy: 0.4744
Epoch 9/500
31/31 [==============================] - 0s 8ms/step - loss: 8.9131 - accuracy: 0.4724
Epoch 10/500
31/31 [==============================] - 0s 2ms/step - loss: 8.5222 - accuracy: 0.4737
E

In [12]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 459us/step
[1.5816140e+00 1.2681772e+00 1.2499341e+01 1.1268976e+01 4.4569564e+00
 4.0100331e+00 5.0434899e+00 4.6567206e+00 1.0020402e+01 1.0619352e+01
 1.4158068e+00 1.7956661e+00 1.7884776e-02 2.9662102e-03]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 5

In [13]:
n=5
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [14]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 421us/step - loss: 26.6360 - accuracy: 0.3991
Epoch 2/500
31/31 [==============================] - 0s 421us/step - loss: 15.1017 - accuracy: 0.4679
Epoch 3/500
31/31 [==============================] - 0s 375us/step - loss: 12.7184 - accuracy: 0.4724
Epoch 4/500
31/31 [==============================] - 0s 409us/step - loss: 11.2396 - accuracy: 0.4672
Epoch 5/500
31/31 [==============================] - 0s 404us/step - loss: 10.1577 - accuracy: 0.4679
Epoch 6/500
31/31 [==============================] - 0s 352us/step - loss: 9.4273 - accuracy: 0.4672
Epoch 7/500
31/31 [==============================] - 0s 389us/step - loss: 9.0172 - accuracy: 0.4711
Epoch 8/500
31/31 [==============================] - 0s 350us/step - loss: 8.7740 - accuracy: 0.4659
Epoch 9/500
31/31 [==============================] - 0s 388us/step - loss: 8.6394 - accuracy: 0.4731
Epoch 10/500
31/31 [==============================] - 0s 492us/step - loss: 8.7031 - a

In [15]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 449us/step
[ 1.8417939   1.6259985  14.371271   12.4827385   5.5246415   4.138823
  4.7573347   4.921271   10.710564   11.247349    1.2253772   1.6877272
  0.24462396  0.12307643]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n = 10

In [16]:
n=10
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [17]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 535us/step - loss: 29.5874 - accuracy: 0.3699
Epoch 2/500
31/31 [==============================] - 0s 368us/step - loss: 14.8811 - accuracy: 0.4646
Epoch 3/500
31/31 [==============================] - 0s 482us/step - loss: 12.2165 - accuracy: 0.4685
Epoch 4/500
31/31 [==============================] - 0s 681us/step - loss: 10.8102 - accuracy: 0.4685
Epoch 5/500
31/31 [==============================] - 0s 418us/step - loss: 9.8967 - accuracy: 0.4679
Epoch 6/500
31/31 [==============================] - 0s 384us/step - loss: 9.4086 - accuracy: 0.4666
Epoch 7/500
31/31 [==============================] - 0s 2ms/step - loss: 9.1766 - accuracy: 0.4724
Epoch 8/500
31/31 [==============================] - 0s 424us/step - loss: 8.9843 - accuracy: 0.4711
Epoch 9/500
31/31 [==============================] - 0s 393us/step - loss: 8.7399 - accuracy: 0.4692
Epoch 10/500
31/31 [==============================] - 0s 363us/step - loss: 8.8180 - accu

In [18]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 644us/step
[ 1.7043934   1.5111862  13.442229   12.448474    4.809536    5.1143994
  5.671726    5.0104036  11.229268   12.914559    1.3650393   2.0399482
  0.14014773 -0.02907469]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n=15

In [19]:
n=15
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [20]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 470us/step - loss: 24.5610 - accuracy: 0.4400
Epoch 2/500
31/31 [==============================] - 0s 456us/step - loss: 13.6948 - accuracy: 0.4692
Epoch 3/500
31/31 [==============================] - 0s 408us/step - loss: 11.7544 - accuracy: 0.4744
Epoch 4/500
31/31 [==============================] - 0s 416us/step - loss: 10.6301 - accuracy: 0.4705
Epoch 5/500
31/31 [==============================] - 0s 377us/step - loss: 9.8777 - accuracy: 0.4757
Epoch 6/500
31/31 [==============================] - 0s 370us/step - loss: 9.4292 - accuracy: 0.4705
Epoch 7/500
31/31 [==============================] - 0s 428us/step - loss: 9.1415 - accuracy: 0.4724
Epoch 8/500
31/31 [==============================] - 0s 929us/step - loss: 8.9373 - accuracy: 0.4757
Epoch 9/500
31/31 [==============================] - 0s 1ms/step - loss: 8.7835 - accuracy: 0.4718
Epoch 10/500
31/31 [==============================] - 0s 520us/step - loss: 8.6919 - accu

In [21]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 3ms/step
[ 1.6167194   0.8287675  15.003606    8.946282    5.0130825   2.34754
  6.3124824   4.3868723  11.191496   11.284431    1.5398586   1.1919498
  0.06831183  0.09557113]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


### n=20

In [22]:
n=20
X = n_components[n][['pca'+ str(x) for x in range(n)]]
Y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=629) 

In [23]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(32, activation='relu', input_dim=len(X_train.columns)),
	tf.keras.layers.Dense(128, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE)
])
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")
model.fit(X_train, y_train, epochs=500, batch_size=50);

Epoch 1/500
31/31 [==============================] - 0s 455us/step - loss: 26.2090 - accuracy: 0.4186
Epoch 2/500
31/31 [==============================] - 0s 423us/step - loss: 13.6336 - accuracy: 0.4666
Epoch 3/500
31/31 [==============================] - 0s 410us/step - loss: 11.5453 - accuracy: 0.4679
Epoch 4/500
31/31 [==============================] - 0s 818us/step - loss: 10.5106 - accuracy: 0.4666
Epoch 5/500
31/31 [==============================] - 0s 434us/step - loss: 9.8231 - accuracy: 0.4692
Epoch 6/500
31/31 [==============================] - 0s 1ms/step - loss: 9.4103 - accuracy: 0.4666
Epoch 7/500
31/31 [==============================] - 0s 3ms/step - loss: 9.2367 - accuracy: 0.4698
Epoch 8/500
31/31 [==============================] - 0s 605us/step - loss: 9.0030 - accuracy: 0.4685
Epoch 9/500
31/31 [==============================] - 0s 404us/step - loss: 8.8262 - accuracy: 0.4692
Epoch 10/500
31/31 [==============================] - 0s 794us/step - loss: 8.8398 - accura

In [24]:
y_hat = model.predict(X_test)
print(y_hat[1])
y_test.head()

13/13 [==============================] - 0s 368us/step
[ 0.51878834  1.4047159   8.342749   17.751005    2.1595416   5.6928353
  3.8907902   7.750232   13.811236   14.350753    2.084751    1.8184897
  0.2090088   0.03180099]


,home_goals,away_goals,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_corners,away_corners,home_fouls,away_fouls,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards
1489,1,5,13,13,5,7,7,3,4,6,0,1,0,0
643,0,0,21,8,4,1,11,7,11,9,4,3,0,0
1263,1,2,9,17,3,7,4,11,8,12,0,0,0,0
848,1,3,19,15,6,6,5,4,9,10,0,0,0,0
95,0,4,7,24,1,8,4,4,14,10,4,2,0,0


The above experiments show us the accuracy of each model increases with an increased number of features up to a point, it seem somewhere between 15 and 20 features, where the accuracy on the training data peaks. I will use 15 principal components going forward for the investigations, as this seems to yield the highest accuracy of those tested here

In terms of predicting exact values for the match facts, I am not expecting exact matches between model and the actual results, just hoping for a close approximation, or at the very least, the correct side with the higher value, i.e. home team scores more goals than the away team is correct

## 15 PC model build

#### Splitting data

In [25]:
X = combined[pure_stats_columns_no_minutes]
y = combined[output_columns]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=543)
print(X_train)

         goals   assists  non_penalty_goals  penalties_scored  \
301  -0.094875 -0.220930          -0.108935          0.014060   
655   1.788608  0.104947           0.194394         -0.115247   
1291  2.641128  0.292058           0.524404          0.105415   
887  -1.201986 -0.302613          -0.124839         -0.038357   
172  -0.174855  0.055731          -0.087689         -0.087167   
...        ...       ...                ...               ...   
132  -0.353935 -0.144334          -0.261522         -0.092413   
479   0.357522  0.089684           0.095154          0.057074   
561   0.382126  0.222171           0.191594          0.139803   
103  -0.438475 -0.195377          -0.396090         -0.042385   
1785  2.166620  0.334845           0.508931          0.099413   

      penalties_attempted  yellow_cards  red_cards  expected_goals  \
301              0.014060      0.317199   0.007532       -0.070804   
655             -0.120927      0.260088   0.009259        0.056842   
1291     

#### Scaling X and y

In [26]:
whole_x_scaler = StandardScaler(copy=True).fit(X)
whole_x_train = whole_x_scaler.transform(X)
dump(whole_x_scaler, '../prediction_scaler.bin')


# y_scaler = StandardScaler(copy=True).fit(y_train)

X_scaler = StandardScaler(copy=True).fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


# y_train = y_scaler.transform(y_train)
# y_test = y_scaler.transform(y_test)

#### Applying PCA

In [27]:
n=15
print(X_train)
pca = PCA(n_components = n, random_state=576)
pca.fit(X)
feature_to_pc_map = pd.DataFrame(pca.components_, columns=pure_stats_columns_no_minutes)
feature_to_pc_map.to_csv("../feature_to_15_pcs.csv")

dump(pca, '../prediction_pca.bin')
whole_X_pca = pca.transform(whole_x_train)

pca = PCA(n_components = n, random_state=576)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)


feature_to_pc_map

[[ 0.0031677  -0.66551577 -0.21848639 ...  0.26402707 -0.21099546
   0.84080285]
 [ 0.77367146  0.36703043  0.48064944 ... -0.90530913  0.68286476
   0.02056297]
 [ 1.12242413  0.95989472  1.24128372 ...  0.21158228  1.07150401
   0.92157332]
 ...
 [ 0.19830155  0.73845591  0.47419645 ...  0.43956876  0.4350281
  -0.75557269]
 [-0.13739354 -0.58455231 -0.88034264 ... -0.13458965  0.9829622
   0.02056297]
 [ 0.92831039  1.09546434  1.20561868 ...  0.82841042  1.07906574
  -0.17496408]]


/opt/homebrew/anaconda3/envs/pl-stats/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
0,0.000742,0.000107,0.000126,0.000011,0.000014,0.000180,0.000007,0.000135,0.000124,0.000106,...,0.002635,0.000061,0.000249,0.000798,0.000553,0.000055,0.000024,0.000019,0.000004,0.000001
1,0.003308,0.000432,0.000617,0.000062,0.000075,0.000334,0.000013,0.000654,0.000594,0.000399,...,0.001258,-0.000159,-0.003251,-0.010252,-0.007047,-0.000734,-0.000302,-0.000229,-0.000055,-0.000018
2,0.004622,0.000648,0.001167,0.000053,0.000067,0.000119,0.000005,0.001108,0.001054,0.000620,...,-0.007540,0.000025,0.002574,0.008196,0.005666,0.000545,0.000242,0.000179,0.000045,0.000018
3,0.012035,0.002059,0.002717,0.000446,0.000586,0.002456,0.000034,0.003237,0.002774,0.002193,...,0.026119,0.000107,-0.000971,-0.003592,-0.002623,-0.000561,-0.000194,-0.000161,-0.000007,-0.000026
4,0.014204,0.001979,0.003664,0.000657,0.000886,-0.000842,0.000064,0.003559,0.002865,0.001999,...,-0.069328,-0.000414,0.005846,0.018834,0.013196,0.001953,0.000472,0.000362,0.000071,0.000039
5,0.001810,-0.003189,-0.001464,-0.000462,-0.000591,0.003951,0.000184,-0.000490,-0.000022,-0.002757,...,0.143571,0.001178,-0.001420,-0.003169,-0.001832,-0.000209,0.000238,0.000252,-0.000037,0.000023
6,-0.034137,-0.003507,-0.006558,-0.000563,-0.000637,0.003178,0.000323,-0.006535,-0.006038,-0.002613,...,0.058445,-0.000214,-0.010432,-0.034739,-0.024413,-0.004918,-0.001544,-0.001385,-0.000120,-0.000039
7,0.025930,0.005495,0.004959,0.000838,0.001058,0.001815,0.000014,0.005545,0.004706,0.005094,...,0.094201,0.000428,-0.008366,-0.026321,-0.018031,-0.003525,-0.000623,-0.000496,-0.000154,0.000027
8,0.042055,0.001819,0.011712,0.001616,0.001821,-0.002527,-0.000161,0.012999,0.011560,0.001821,...,0.078699,-0.002135,-0.028502,-0.090915,-0.062872,-0.005716,-0.003292,-0.002819,-0.000400,-0.000073
9,0.068752,0.000388,0.016361,0.002341,0.002621,0.004623,0.000151,0.019555,0.017466,-0.001020,...,-0.049732,0.001767,0.034510,0.102680,0.068847,0.004220,0.003356,0.002518,0.000605,0.000233


The below table shows the breakdown of each principle component in terms of the proportion of the value from each of the table columns used to make it up

In [28]:
df = pd.DataFrame(pca.components_, columns=pure_stats_columns_no_minutes)
df.index.name = "principle_component_number"
df

,goals,assists,non_penalty_goals,penalties_scored,penalties_attempted,yellow_cards,red_cards,expected_goals,non_penalty_expected_goals,expected_assisted_goals,...,clearances,errors_leading_to_shot,goals_against,shots_on_target_against,saves,clean_sheets,penalties_faced,penalties_allowed,penalties_saved,penalties_missed
principle_component_number,,,,,,,,,,,,,,,,,,,,,
0,-0.114827,-0.129052,-0.116702,-0.067303,-0.068918,-0.133266,-0.078929,-0.119126,-0.120603,-0.134648,...,-0.115752,-0.110908,-0.055217,-0.056287,-0.056471,-0.050583,-0.051109,-0.050263,-0.037204,-0.027362
1,-0.076207,-0.074874,-0.096449,-0.099214,-0.100669,0.002926,0.017051,-0.102086,-0.096433,-0.071295,...,0.037691,0.155209,0.299717,0.302302,0.301513,0.281103,0.297094,0.282476,0.247426,0.170881
2,0.129684,0.094786,0.202589,0.201222,0.203529,-0.091492,-0.078626,0.197089,0.184715,0.083927,...,-0.199649,-0.070431,0.147101,0.148932,0.149239,0.149440,0.149437,0.141045,0.125463,0.091498
3,0.012339,-0.056923,0.047812,0.517905,0.509710,0.044948,-0.002697,0.115383,0.042584,-0.049077,...,0.120272,0.070535,-0.014020,-0.015480,-0.015441,-0.007412,-0.033999,-0.026027,-0.035921,-0.050466
4,0.393711,-0.057483,0.078656,0.006452,-0.016994,0.072440,-0.070190,0.100952,0.114822,-0.048546,...,0.052095,0.001662,-0.029240,-0.029546,-0.031542,-0.026292,0.011700,0.004464,0.043552,-0.007951
5,0.065291,0.118182,-0.177586,0.105684,0.110093,-0.096678,-0.108414,-0.176219,-0.214140,0.138487,...,-0.280313,-0.106535,0.047202,0.042639,0.040278,0.033782,0.012787,0.051564,0.025091,-0.300169
6,0.027925,0.089704,-0.001078,-0.024269,-0.035036,-0.091844,0.046270,-0.024446,-0.021329,0.053927,...,-0.046283,-0.007501,-0.111597,-0.071465,-0.052535,0.029501,-0.072881,-0.161501,-0.174140,0.824104
7,-0.060446,0.059229,0.035370,-0.041391,-0.056498,-0.041428,-0.863648,0.018545,0.030034,0.034526,...,0.116808,0.017207,-0.017978,-0.015927,-0.013996,-0.006914,-0.022590,0.023737,-0.142707,-0.107514
8,-0.015947,-0.021876,-0.129169,0.147798,0.148286,0.112371,-0.392915,-0.066364,-0.098471,0.005349,...,0.048611,0.029535,0.014670,0.026517,0.031168,-0.018400,0.064582,0.028775,-0.000350,0.369304


#### Model Definition

In [29]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(output_columns)

Define

In [30]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Dense(15, activation='relu'),
	tf.keras.layers.Dense(12, activation='relu'),
	tf.keras.layers.Dense(OUTPUT_SIZE, activation='relu')
])

Compile

In [31]:
model.compile(loss="mean_squared_error", optimizer="sgd", metrics="accuracy")

Fit

In [32]:
model.fit(X_train, y_train, epochs=500, batch_size=25);

Epoch 1/500
62/62 [==============================] - 0s 435us/step - loss: 24.9812 - accuracy: 0.3212
Epoch 2/500
62/62 [==============================] - 0s 448us/step - loss: 12.0798 - accuracy: 0.4776
Epoch 3/500
62/62 [==============================] - 0s 362us/step - loss: 10.1604 - accuracy: 0.4802
Epoch 4/500
62/62 [==============================] - 0s 522us/step - loss: 9.6588 - accuracy: 0.4783
Epoch 5/500
62/62 [==============================] - 0s 343us/step - loss: 9.0850 - accuracy: 0.4770
Epoch 6/500
62/62 [==============================] - 0s 438us/step - loss: 9.1704 - accuracy: 0.4802
Epoch 7/500
62/62 [==============================] - 0s 365us/step - loss: 9.0556 - accuracy: 0.4737
Epoch 8/500
62/62 [==============================] - 0s 351us/step - loss: 8.8669 - accuracy: 0.4763
Epoch 9/500
62/62 [==============================] - 0s 358us/step - loss: 8.8333 - accuracy: 0.4789
Epoch 10/500
62/62 [==============================] - 0s 331us/step - loss: 8.7631 - acc

#### Testing

In [33]:
y_hat = model.predict(X_test)

13/13 [==============================] - 1s 494us/step


In [34]:
total_shots = 0
total_shots_over = 0
total_shots_under = 0

total_shots_on_target = 0
total_shots_on_target_over = 0
total_shots_on_target_under = 0

total_booking_points = 0
total_booking_points_over = 0
total_booking_points_under = 0

correct_score = 0

winner = 0

total_corners = 0
total_corners_over = 0
total_corners_under = 0

total_fouls = 0
total_fouls_over = 0
total_fouls_under = 0

goals_over = 0
goals_under = 0

all_under = 0
all_over = 0

total_tested  = len(y_hat)

In [35]:
for idx, y in enumerate(y_hat):
	home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
	home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = y_test.iloc[idx].tolist()
	
	total_shots += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) == home_shots+away_shots else 0
	total_shots_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots else 0
	total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
	
	total_shots_on_target += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) == home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_over += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target else 0
	total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0

	total_booking_points += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 == (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_over += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
	total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0

	correct_score += 1 if np.floor(home_goals_hat) == home_goals and np.floor(away_goals_hat) == away_goals else 0
	
	winner += 1 if (home_goals_hat > away_goals_hat and home_goals > away_goals) or (home_goals_hat < away_goals_hat and home_goals < away_goals) else 0

	total_fouls += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) == home_fouls + away_fouls else 0
	total_fouls_over += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) > home_fouls+ away_fouls else 0
	total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0

	total_corners += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) == home_corners + away_corners else 0
	total_corners_over += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners else 0
	total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0

	goals_over += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0
	goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

	all_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0
	all_over += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) > home_shots+away_shots and np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) > home_shots_on_target+away_shots_on_target and (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 > (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 and np.floor(home_corners_hat) + np.floor(away_corners_hat) > home_corners + away_corners and np.floor(home_goals_hat) + np.floor(away_goals_hat) > home_goals + away_goals else 0

In [36]:
print("total_shots: "+str(total_shots/total_tested))
print("total_shots_over: "+str(total_shots_over/total_tested))
print("total_shots_under: "+str(total_shots_under/total_tested))
print("\n")
print("total_shots_on_target: "+str(total_shots_on_target/total_tested))
print("total_shots_on_target_over: "+str(total_shots_on_target_over/total_tested))
print("total_shots_on_target_under: "+str(total_shots_on_target_under/total_tested))
print("\n")
print("total_booking_points: "+str(total_booking_points/total_tested))
print("total_booking_points_over: "+str(total_booking_points_over/total_tested))
print("total_booking_points_under: "+str(total_booking_points_under/total_tested))
print("\n")
print("correct_score: "+str(correct_score/total_tested))
print("\n")
print("winner: "+str(winner/total_tested))
print("\n")
print("total_corners: "+str(total_corners/total_tested))
print("total_corners_over: "+str(total_corners_over/total_tested))
print("total_corners_under: "+str(total_corners_under/total_tested))
print("\n")
print("total_fouls: "+str(total_fouls/total_tested))
print("total_fouls_over: "+str(total_fouls_over/total_tested))
print("total_fouls_under: "+str(total_fouls_under/total_tested))
print("\n")
print("goals_over: "+str(goals_over/total_tested))
print("goals_under: "+str(goals_under/total_tested))
print("\n")
print("all_over: "+str(all_over/total_tested))
print("all_under: "+str(all_under/total_tested))

total_shots: 0.06994818652849741
total_shots_over: 0.39378238341968913
total_shots_under: 0.5362694300518135


total_shots_on_target: 0.12953367875647667
total_shots_on_target_over: 0.31865284974093266
total_shots_on_target_under: 0.5518134715025906


total_booking_points: 0.18393782383419688
total_booking_points_over: 0.15544041450777202
total_booking_points_under: 0.6606217616580311


correct_score: 0.10362694300518134


winner: 0.4274611398963731


total_corners: 0.13730569948186527
total_corners_over: 0.31088082901554404
total_corners_under: 0.5518134715025906


total_fouls: 0.07253886010362694
total_fouls_over: 0.32642487046632124
total_fouls_under: 0.6010362694300518


goals_over: 0.20725388601036268
goals_under: 0.5621761658031088


all_over: 0.0051813471502590676
all_under: 0.10880829015544041


### Tuning

Create a function to build and compile a model, with the arguments in it being the values to be update during tuning

In [37]:
def get_mlp_model(hidden_layer_one=13, dropout=0.2, learn_rate=0.01, n_h_layers=1):

	model = tf.keras.models.Sequential()

	# input
	model.add(tf.keras.layers.Dense(15, activation="relu", input_dim=15))

	for i in range(n_h_layers):
		model.add(tf.keras.layers.Dense(hidden_layer_one, activation="relu"))

	# dropout layer to remove redundant nodes
	model.add(tf.keras.layers.Dropout(dropout))
	
	# output
	model.add(tf.keras.layers.Dense(14, activation="relu"))

	model.compile(
		optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learn_rate),
		loss="mse",
		metrics=["accuracy"])

	return model

Build a model using the *get_mlp_model* function, letting tensorflow use this function when building the models

In [38]:
model = KerasRegressor(model=get_mlp_model, verbose=0, hidden_layer_one=10, learn_rate=0.01, dropout=0.05, n_h_layers=1);

Define the hyperparameter search space, and then generate the search object to be used later, by RandomizedSearchCV

In [39]:
# define a grid of the hyperparameter search space
hidden_layer_one = [10]
learn_rate = [1e-4]
dropout = [0.3]
batch_size = [64]
epochs = [15]
n_h_layers = [4]

# create a dictionary from the hyperparameter grid
grid = dict(
	hidden_layer_one=hidden_layer_one,
	learn_rate=learn_rate,
	dropout=dropout,
	batch_size=batch_size,
	epochs=epochs,
	n_h_layers=n_h_layers
)

Build a scoring function for the searcher, average the value of all the *under* stats and return this

In [40]:
def scoring(estimator, test_x: np.ndarray, test_y: pd.DataFrame) -> float:
	test_y = test_y.to_numpy()
	
	# estimator.save("../stats_regression_model.h5")
	y_hat = estimator.predict(test_x)
	
	average_under_rate = 0

	total_shots_under = 0
	total_shots_on_target_under = 0
	total_booking_points_under = 0
	total_corners_under = 0
	total_fouls_under = 0
	goals_under = 0

	total_tested = len(y_hat)
	
	for idx, y in enumerate(y_hat):

		home_goals_hat, away_goals_hat, home_shots_hat, away_shots_hat, home_shots_on_target_hat, away_shots_on_target_hat, home_corners_hat, away_corners_hat, home_fouls_hat, away_fouls_hat, home_yellow_cards_hat, away_yellow_cards_hat, home_red_cards_hat, away_red_cards_hat = y
		home_goals, away_goals, home_shots, away_shots, home_shots_on_target, away_shots_on_target, home_corners, away_corners, home_fouls, away_fouls, home_yellow_cards, away_yellow_cards, home_red_cards, away_red_cards = test_y[idx]

		total_shots_under += 1 if np.floor(home_shots_hat)+np.floor(away_shots_hat) < home_shots+away_shots else 0
		total_shots_on_target_under += 1 if np.floor(home_shots_on_target_hat)+np.floor(away_shots_on_target_hat) < home_shots_on_target+away_shots_on_target else 0
		total_booking_points_under += 1 if (np.floor(home_yellow_cards_hat)+np.floor(away_yellow_cards_hat))*10+(np.floor(home_red_cards_hat)+np.floor(away_red_cards_hat))*25 < (home_yellow_cards+away_yellow_cards)*10+(home_red_cards+away_red_cards)*25 else 0
		total_corners_under += 1 if np.floor(home_corners_hat) + np.floor(away_corners_hat) < home_corners + away_corners else 0
		total_fouls_under += 1 if np.floor(home_fouls_hat) + np.floor(away_fouls_hat) < home_fouls+ away_fouls else 0
		goals_under += 1 if np.floor(home_goals_hat) + np.floor(away_goals_hat) < home_goals + away_goals else 0

		average_under_rate += ((total_shots_under/total_tested)+(total_shots_on_target_under/total_tested)+(total_booking_points_under/total_tested)+(total_fouls_under/total_tested)+(total_corners_under/total_tested)+(goals_under/total_tested))/6

	average_under_rate = average_under_rate/total_tested
	
	return average_under_rate

Generate the searcher object, used to iterate over the search parameters and determine the best hyper-parameter setup from the space we've created above

In [41]:
searcher = GridSearchCV(estimator=model, n_jobs=-2, 
	param_grid=grid, scoring=scoring, verbose=4, cv=3, refit=True)

searchResults = searcher.fit(X_train, y_train)

bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("[INFO] best score is {:.2f} using {}".format(bestScore,bestParams))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


2024-01-16 10:09:34.268147: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-16 10:09:34.268524: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-16 10:09:34.269704: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[CV 1/3] END batch_size=64, dropout=0.3, epochs=15, hidden_layer_one=10, learn_rate=0.0001, n_h_layers=4;, score=0.305 total time=   0.7s
[CV 3/3] END batch_size=64, dropout=0.3, epochs=15, hidden_layer_one=10, learn_rate=0.0001, n_h_layers=4;, score=0.383 total time=   0.7s
[CV 2/3] END batch_size=64, dropout=0.3, epochs=15, hidden_layer_one=10, learn_rate=0.0001, n_h_layers=4;, score=0.334 total time=   0.7s
[INFO] best score is 0.34 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}


In [42]:
tuned_model = searcher.best_params_
tuned_model

{'batch_size': 64,
 'dropout': 0.3,
 'epochs': 15,
 'hidden_layer_one': 10,
 'learn_rate': 0.0001,
 'n_h_layers': 4}

In [45]:
hidden_layer_one = [10]
learn_rate = [1e-4]
dropout = [0.3]
batch_size = [64]
epochs = [15]
n_h_layers = [4]

model = get_mlp_model(hidden_layer_one=10, learn_rate=0.0001, dropout=0.3, n_h_layers=4)
model.fit(X_train, y_train, epochs=50, batch_size=64)

model.save("../stats_regression_model.h5")
pd.DataFrame(data=components[:, [p for p in range(15)]], columns=pca.get_feature_names_out(), )

Epoch 1/50
25/25 [==============================] - 0s 475us/step - loss: 39.8470 - accuracy: 0.1830
Epoch 2/50
25/25 [==============================] - 0s 448us/step - loss: 16.6264 - accuracy: 0.4212
Epoch 3/50
25/25 [==============================] - 0s 461us/step - loss: 14.4110 - accuracy: 0.4536
Epoch 4/50
25/25 [==============================] - 0s 532us/step - loss: 12.4516 - accuracy: 0.4685
Epoch 5/50
25/25 [==============================] - 0s 442us/step - loss: 12.4488 - accuracy: 0.4770
Epoch 6/50
25/25 [==============================] - 0s 343us/step - loss: 12.3224 - accuracy: 0.4776
Epoch 7/50
25/25 [==============================] - 0s 388us/step - loss: 11.9960 - accuracy: 0.4744
Epoch 8/50
25/25 [==============================] - 0s 387us/step - loss: 11.9759 - accuracy: 0.4737
Epoch 9/50
25/25 [==============================] - 0s 353us/step - loss: 11.7950 - accuracy: 0.4770
Epoch 10/50
25/25 [==============================] - 0s 399us/step - loss: 11.8954 - accura

,pca0,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,pca11,pca12,pca13,pca14
0,5.106395,-3.114979,-4.900365,0.901282,-1.282006,0.116542,-1.205167,-1.430315,0.958539,1.278368,0.842471,-0.796134,-0.243988,-0.037469,0.736452
1,2.246327,-1.846671,0.441061,0.463044,-0.387274,0.375706,-0.558049,-0.105375,0.284333,0.257541,0.908626,0.162876,-0.460670,-0.260484,-0.169108
2,0.149040,-0.892374,0.393810,1.552950,0.097456,-0.826826,0.439045,0.758487,0.315798,0.569891,0.540703,-1.101917,0.359163,0.104914,0.111324
3,-1.545967,-3.814107,0.322819,0.528597,-0.013403,1.295976,-0.535898,-0.276738,0.924086,-0.523315,0.167971,0.389170,-0.211593,-0.149914,0.327121
4,-0.970680,-0.398294,-4.108531,-1.100442,-0.395523,1.961321,0.219578,0.032103,1.130510,-0.703218,-0.110109,-0.012815,0.524418,-0.509115,-0.601722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1922,-7.786681,0.413605,-0.327300,-1.370923,-0.888390,0.819318,0.010928,0.469753,-0.796626,0.296369,-0.124418,-0.841189,-0.122906,-0.281041,-0.152548
1923,1.555404,2.658409,-2.579289,1.525813,0.147165,0.164834,0.781047,-0.776053,-0.077260,0.120337,-0.348983,0.083507,0.299046,0.127833,0.600224
1924,-0.046183,-2.356918,-0.844679,-1.127035,-1.142690,-0.444876,-0.257349,0.509206,0.722522,0.572677,0.369674,-0.162800,0.144043,-0.107111,0.370080
1925,-5.720248,0.149320,0.667314,-0.729552,-0.678868,0.207523,0.592951,-1.069626,-0.824832,-0.566193,0.172119,-0.128086,-0.316572,0.279469,-0.511047


In [44]:
print(len
	  (X_train))

1541


## Next v  steps

- Tune parameters and layers on the 15 PC model to increase the accuracy
- Create a method to define the accuracy on the test set; strict equality for each of the output values is not necessary as under/over options on the betting platform mean a close approximation of the outcome is all that is required. Success/accuracy measure based on the bet markets available for each game:
	- Total shots
	- Total booking points
	- Correct score
	- Outright
	- Total corners
	- Under/over goals
- Save the model
- Complete the notebook with all annotations and text

RandomSearchCV  
best score is 0.39 using {'n_h_layers': 5, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 100, 'dropout': 0.2, 'batch_size': 64}  
best score is 0.40 using {'n_h_layers': 7, 'learn_rate': 0.0001, 'hidden_layer_one': 15, 'epochs': 1000, 'dropout': 0.75, 'batch_size': 16}  
best score is 0.49 using {'n_h_layers': 3, 'learn_rate': 0.0001, 'hidden_layer_one': 30, 'epochs': 10, 'dropout': 0.3, 'batch_size': 64} 

GridSearchCV  
best score is 0.47 using {'batch_size': 64, 'dropout': 0.4, 'epochs': 100, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 2}  
best score is 0.49100 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 10, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 3}  
best score is 0.49130 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-05, 'n_h_layers': 4}  
best score is 0.49132 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 20, 'hidden_layer_one': 15, 'learn_rate': 1e-06, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 1e-07, 'n_h_layers': 6}  
best score is 0.49138 using {'batch_size': 64, 'dropout': 0.3, 'epochs': 15, 'hidden_layer_one': 10, 'learn_rate': 0.0001, 'n_h_layers': 4}  
best score is 0.49138 using {'batch_size': 32, 'dropout': 0.2, 'epochs': 12, 'hidden_layer_one': 10, 'learn_rate': 1e-05, 'n_h_layers': 4}  

- During
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.459 total time=   4.3s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.462 total time=   3.2s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.430 total time=   3.5s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=1;, score=0.458 total time=   5.3s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.412 total time=   3.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=2;, score=0.451 total time=   3.5s
	- [CV 1/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.447 total time=   3.8s
	- [CV 2/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.413 total time=   4.8s
	- [CV 3/3] END batch_size=64, dropout=0.2, epochs=100, hidden_layer_one=15, learn_rate=0.0001, n_h_layers=3;, score=0.415 total time=   4.9s

## Questions

- Scaling y reduces the accuracy of the model by a factor of 6, why?